In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Input, AveragePooling1D, Dropout, Softmax
from keras.optimizers import Adam, Nadam
from keras.regularizers import L2
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, Callback, LambdaCallback
from keras.metrics import Metric
from tensorflow import keras

from time import perf_counter
import functools

In [2]:
stresses = ['Gm', 'Drought', 'Nutrient_Deficiency', 'Fs', 'Salinity']

In [3]:
csv_path = r'..\combined.csv'
df = pd.read_csv(csv_path)
df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')
df.drop(columns=['Fungal_infection'], inplace=True, errors='ignore')
df[stresses] = df[stresses].astype(bool)

C:\Users\amitt\AppData\Local\Temp\ipykernel_5344\1297277926.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [4]:
spec_cols = [col for col in df.columns if col[0] == 'X']

In [5]:
fuzzy_win = 5

In [6]:
def cnn_reshape(x):
    return x.reshape((-1, x.shape[1], 1))

In [7]:
blur_factor = 4

# x = drop_res(fuzzy_dx3, blur_factor=blur_factor)
x = df[spec_cols].values
y = df[stresses].values

x /= x.max()

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2)

In [40]:
class Accuracy(Metric):
    def __init__(self, name='accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_variable(
            shape=(),
            initializer='zeros',
            name='accuracy'
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        p = tf.cast(tf.reduce_all((y_pred > .5) == y_true, axis=1), tf.float32)
        
        self.accuracy.assign(tf.reduce_mean(p))

    def result(self):
        return self.accuracy

In [77]:
@functools.cache
def fuzzy_dx_init(shape, dtype=None):
    half_shape = list(shape)
    half_shape[0] //= 2
    half_shape = tuple(half_shape)
    return np.vstack((np.ones(half_shape) * -1/half_shape[0], np.ones(half_shape)/half_shape[0]))

model = Sequential([
    Input(shape=(x_train.shape[1],1)),
    Conv1D(1, fuzzy_win*2, trainable=False, kernel_initializer=fuzzy_dx_init),
    Conv1D(1, fuzzy_win*2, trainable=False, kernel_initializer=fuzzy_dx_init),
    Conv1D(1, fuzzy_win*2, trainable=False, kernel_initializer=fuzzy_dx_init),
    AveragePooling1D(blur_factor),
    Conv1D(20, 30, kernel_regularizer=L2(.001), name='conv1', activation='relu'),
    Conv1D(20, 30, kernel_regularizer=L2(.001), name='conv2', activation='relu'),
    MaxPooling1D(2),
    Dropout(.02),
    Flatten(),
    Dense(100, kernel_regularizer=L2(.001), activation='relu'),
    Dropout(.02),
    Dense(100, kernel_regularizer=L2(.001), activation='relu'),
    Dropout(.02),
    Dense(100, kernel_regularizer=L2(.001), activation='relu'),
    Dropout(.02),
    Dense(100, kernel_regularizer=L2(.001), activation='relu'),
    Dropout(.02),
    Dense(10, kernel_regularizer=L2(.001), activation='relu'),
    Dropout(.02),
    Dense(y.shape[1], activation='sigmoid'),
])

model.compile(optimizer=Nadam(1e-4), loss='binary_crossentropy', metrics=[Accuracy])

In [ ]:
start = perf_counter()

history = model.fit(
    cnn_reshape(x_train),
    y_train,
    epochs=300,
    validation_data=(cnn_reshape(x_val), y_val),
    batch_size=80,
)

print(perf_counter() - start)

Epoch 1/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.0612 - loss: 1.1152 - val_accuracy: 0.0000e+00 - val_loss: 0.8774
Epoch 2/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.1369 - loss: 0.8381 - val_accuracy: 0.4167 - val_loss: 0.6351
Epoch 3/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.3593 - loss: 0.6336 - val_accuracy: 0.4167 - val_loss: 0.5621
Epoch 4/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.3624 - loss: 0.5729 - val_accuracy: 0.4167 - val_loss: 0.5282
Epoch 5/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.3636 - loss: 0.5383 - val_accuracy: 0.4167 - val_loss: 0.5046
Epoch 6/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.3654 - loss: 0.5104 - val_accuracy: 0.4167 - val_loss: 0.4918
Epoch 7/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.3696 - loss: 0.4978 - val_accuracy: 0.4167 - val_loss: 0.4791
Epoch 8/300
237/237 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.3677 - loss: 0.485

In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.plot(history.history['loss'], label='Training')
ax.plot(
    np.convolve(np.array(history.history['val_loss']), np.ones(10)/10, mode='valid'),
    label='Validation (Running Mean)'
)
ax.legend()
ax.set_xlabel('# Epochs')
ax.set_ylabel('Loss')
ax.set_ylim((0, 1))
ax.set_title('CNN Loss over Training')

In [ ]:
y_pred = model.predict(cnn_reshape(x_val))